# 1.LLMを導入してみよう

## 1.1.必要なライブラリのインストール

In [ ]:
!pip install langchain langchain-openai gradio python-dotenv langchain-community

## 1.2.ChatGPTのAPIキーの登録

In [ ]:
# APIキーの登録
import os
os.environ['OPENAI_API_KEY'] = 'yyHDWJTRyxJBtgsEIE3Y7_BLZOHs5XDCVr0LfZQrzoOJ8cjsFqoy4-lYb5orGEwrH3OrwDEDq2wDc3TEJauWCHA'

# openai_api_base="https://api.openai.com/v1/"
openai_api_base="https://api.openai.iniad.org/api/v1"

## 1.3.Chatbotの起動

In [ ]:
import os
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import HumanMessage, AIMessage
from langchain.callbacks import get_openai_callback
import warnings
warnings.filterwarnings("ignore")

# 固定パラメータ設定
MODEL_NAME = "gpt-4o-mini"
TEMPERATURE = 0.7

class OpenAIChatbot:
    def __init__(self):
        # APIキーを環境変数から取得
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("OPENAI_API_KEYが環境変数に設定されていません。")

        # グローバル変数として宣言された openai_api_base を使用
        global openai_api_base
        print(f"✅ OpenAI Base URL: {openai_api_base}")

        # モデル初期化
        self.llm = ChatOpenAI(
            model=MODEL_NAME,
            temperature=TEMPERATURE,
            base_url=openai_api_base,
            openai_api_key=api_key,
            streaming=False
        )

        self.memory = ConversationBufferWindowMemory(
            k=10,
            return_messages=True
        )

        self.system_prompt = """あなたは親切で知識豊富なAIアシスタントです。
ユーザーの質問に対して、正確で有用な情報を提供してください。
わからないことがあれば、素直に「わからない」と答えてください。
日本語で自然な会話を心がけてください。"""

    def chat(self, user_input):
      messages = [{"role": "system", "content": self.system_prompt}]
      for msg in self.memory.chat_memory.messages:
          if isinstance(msg, HumanMessage):
              messages.append({"role": "user", "content": msg.content})
          elif isinstance(msg, AIMessage):
              messages.append({"role": "assistant", "content": msg.content})
      messages.append({"role": "user", "content": user_input})

      try:
          with get_openai_callback() as cb:
              response = self.llm.invoke(messages)
          self.memory.chat_memory.add_user_message(user_input)
          self.memory.chat_memory.add_ai_message(response.content)
          return response.content
      except Exception as e:
          print(f"❌ Error during chat: {e}")
          return f"⚠️ エラーが発生しました: {e}"


# インスタンス生成
chatbot = OpenAIChatbot()

def chat_response(message, history):
    return chatbot.chat(message)

# Gradio Chat UI のみ
def create_ui():
    return gr.ChatInterface(
        fn=chat_response,
        title="ChatBot（ChatGPT利用）",
        description="OpenAIによる日本語チャットアシスタントです。",
        examples=[
            "INIAD工業株式会社の就業規定について教えてください",
            "装置Aの停止時の適切な対応方法について教えてください"
        ],
        submit_btn="送信",
        stop_btn="停止",
    )

# 実行
if __name__ == "__main__":
    demo = create_ui()
    demo.launch()


# 2.RAGを導入してみよう
このノートブックでは、LangChainとChromaを使ってPDFドキュメントから情報を取り込み、チャンク化し、OpenAIの埋め込みモデルを使ってベクトルストアを構築し、簡単な質問応答（RAG）を実践します。

*動作環境: Google Colabを想定しています。*

---

## 2.1. ライブラリのインストール
必要なパッケージをインストールします。


In [ ]:
!pip install --upgrade pip
!pip install langchain chromadb langchain-chroma langchain-openai pypdf tiktoken langchain-community requests gradio

## 2.2. 必要なモジュールのインポート
必要なモジュールを読み込みます。


In [ ]:
import os
import zipfile
import requests
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.chains import RetrievalQA
import gradio as gr
from langchain.prompts import PromptTemplate

## 2.3. PDFドキュメントのダウンロード


In [ ]:
!wget -O rag_sample.pdf "https://www.dl.dropboxusercontent.com/scl/fi/5q5wgjugmc7v0vc4youde/.pdf?rlkey=3mudbfkqjb7xj86hffmh962v1&st=toe9octw&dl=1"


## 2.4. ドキュメントの読み込みとチャンク化
PDFを読み込み、長いテキストを扱いやすいサイズに分割します。


In [ ]:
# PDFローダーでドキュメントを取得
loader = PyPDFLoader("/content/rag_sample.pdf")
docs = loader.load()

# チャンク化の設定（例: 1000文字のチャンク、重複200文字）
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)
chunks = splitter.split_documents(docs)
print(f"Created {len(chunks)} chunks.")

for chunk in chunks:
  print(f"====================================")
  print(chunk)
  print(f"====================================")

## 2.5. チャンクのベクトル化とベクトルの保存
OpenAIEmbeddingモデルでテキストチャンクを埋め込み、Chromaに保存します。

In [ ]:
# 埋め込みモデルの初期化
embeddings = OpenAIEmbeddings(
    openai_api_base=openai_api_base,
    openai_api_key=os.environ['OPENAI_API_KEY']
)
# Chromaベクトルストアを作成（ローカルに永続化）
vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory="./chroma_db/rag_sample")

# チャンクをベクトルDBに追加
vectorstore.add_documents(chunks)

## 2.6.ベクトル検索をしてみよう

In [ ]:
# 質問の例
query = "INIAD工業株式会社の概要について教えてください"

# ベクトル検索
docs = vectorstore.similarity_search(query, k=1)

# 結果を表示
for i, doc in enumerate(docs, 1):
    print(f"[{i}] {doc.metadata.get('source', '')}")
    print(doc.page_content[:300], "\n")

## 2.7. RAGの作成


In [ ]:
# 生成モデルの初期化
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.0,
    openai_api_base=openai_api_base,
    openai_api_key=os.environ['OPENAI_API_KEY']
)

# QAチェーンの設定
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

## 2.8. RAGの実行
実際にユーザーが質問を入力して回答を得ます。

In [ ]:
print("============================= RAGの回答 =============================")
query = "INIAD工業株式会社の概要について教えてください"
rag_answer = qa.invoke(query)
print(rag_answer["result"])

print("============================= LLM(ChatGPT)の回答 =============================")
llm_answer = llm.invoke(query)
print(llm_answer.content)

## 2.9.ドキュメントをまとめて処理する

### 2.9.1. ドキュメントのダウンロード

In [ ]:
# === 設定 ===
dropbox_url = "https://www.dropbox.com/scl/fo/53hp5xxh8jo08t8ckhvyf/APEhI4MajStJbSYZClqvK0k?rlkey=4s5bwt71v39tfp4v075f4xi1l&st=n8dnqbjz&dl=1"  # 直リンク化
zip_path = "rag_handson.zip"
extract_dir = "rag_handson"

# === 1. DropboxからZIPをダウンロード ===
print("Downloading ZIP from Dropbox...")
with requests.get(dropbox_url, stream=True) as r:
    r.raise_for_status()
    with open(zip_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
print("Download complete.")

# === 2. ZIPファイルを解凍 ===
print("Extracting ZIP...")
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)
print("Extraction complete.")

### 2.9.2. ダウンロードしたファイルを一括でベクトルDBに保存する

In [ ]:
# === 1. PDF読み込み & チャンク化 ===
docs = []
for path in Path(extract_dir).rglob("*.pdf"):
    try:
        loader = PyPDFLoader(str(path))
        docs.extend(loader.load())
    except Exception as e:
        print(f"❌ {path.name} の読み込みに失敗しました: {e}")


print(f" {len(docs)} ページのPDFを読み込みました。")

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = splitter.split_documents(docs)
print(f" {len(chunks)} 個のチャンクに分割されました。")

# === 2. 埋め込みとベクトルストアの作成 ===
embeddings = OpenAIEmbeddings(
    openai_api_base=openai_api_base,
    openai_api_key=os.environ['OPENAI_API_KEY']
)
vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory="./chroma_db/rag_handson"
)
vectorstore.add_documents(chunks)
print("ベクトルDBに保存が完了しました。")

## 2.10. Chatbot（RAG）の起動

In [ ]:
# --- 埋め込みモデルとベクターストア初期化 ---
embeddings = OpenAIEmbeddings(
    openai_api_base=openai_api_base,
    openai_api_key=os.environ['OPENAI_API_KEY']
)
vectorstore = Chroma(
    persist_directory="./chroma_db/rag_handson",
    embedding_function=embeddings
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})  # 上位3件を返す


# --- LLM 初期化 ---
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.0,
    base_url=openai_api_base,
    openai_api_key=os.environ.get("OPENAI_API_KEY")
)


# --- カスタムプロンプト ---
template = """以下はコンテキストと質問です。
コンテキストのみを使って質問に答えてください。
わからなければ「わかりません」と答えてください。

コンテキスト:
{context}

質問:
{question}
"""
prompt = PromptTemplate.from_template(template)


# --- RetrievalQA チェーンの作成（ソース取得付き） ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)


# --- 回答関数（ドキュメントも含める） ---
def chat_response(message, history):
    try:
        result = qa_chain.invoke({"query": message})
        answer = result.get("result", "申し訳ありませんが、わかりません。")
        sources = result.get("source_documents", [])

        if sources:
            formatted_sources = []
            for i, doc in enumerate(sources):
                title = doc.metadata.get("source", f"ドキュメント{i+1}")
                snippet = doc.page_content.strip().replace("\n", " ").replace("　", " ")
                snippet = snippet[:300] + ("…" if len(snippet) > 300 else "")
                formatted_sources.append(f"""
【{i+1}. {title}】
{snippet}
""")

            sources_text = "\n---\n".join(formatted_sources)
            full_response = f"{answer}\n\n📄 **参照ドキュメント**\n{sources_text}"
        else:
            full_response = answer

        return full_response

    except Exception as e:
        import traceback
        traceback.print_exc()
        return f"エラーが発生しました: {str(e)}"


# --- Gradio UI ---
def create_ui():
    return gr.ChatInterface(
        fn=chat_response,
        title="ChatBot（RAG + ソース表示付き）",
        description="PDFなどから構築されたベクターストアを用いた質問応答。参照元も表示します。",
        examples=[
            "INIAD工業株式会社の就業規定について教えてください",
            "装置Aの停止時の適切な対応方法について教えてください"
        ],
        submit_btn="送信",
        stop_btn="停止",
        type="messages"
    )

# --- 実行 ---
if __name__ == "__main__":
    demo = create_ui()
    demo.launch()